In [72]:
import re
import string
import sys
import os
import urllib
from bs4 import BeautifulSoup
import requests
import shutil
import time
from lxml import etree

In [73]:
#Python 3 uses utf-8, so you have no need to use reload(sys)
#I have two questions, the first one is, could I caculate the page numbers 
#The second one is, could i download all pictures from the weibo
user_id = 1942473263
cookie = {"Cookie": "_T_WM=8da5d2618b8c46398dd6eefbeb41bf3a; SUB=_2A2533ArPDeRhGeBO41cU-SjLyzuIHXVVPpaHrDV6PUJbkdBeLVLjkW1NRaV9ki5ImY17xTMw5ERzwxkZxHlAty9B; SUHB=0C1ygj5mUjk3v9; SCF=AktKW5we8Aunj-tXB6mFFTmknwn-c0nN2Jm0SaESeB8IkG3a-G6CYlO5f0WNLYS3b-wNFAgjb6YmUzU4VrfhqZQ.; SSOLoginState=1524136607; M_WEIBOCN_PARAMS=uicode%3D20000174"}
url = 'http://weibo.cn/u/%d?filter=1&page=1'%user_id
html = requests.get(url, cookies = cookie).content

#XML is an inherently hierarchical data format, and the most natural way to represent it is with a tree
selector = etree.HTML(html)
pageNum = (int)(selector.xpath('//input[@name="mp"]')[0].attrib['value'])

result = ""
urllist_set = set()
word_count = 1
image_count = 1

print(u'ready')
print(pageNum)

ready
172


In [74]:
#刷新输出
sys.stdout.flush()

times = 5
one_step = pageNum/times

for step in range(times):
    if step < times - 1:
        i = step * one_step + 1
        j = (step + 1) * one_step + 1
    else:
        i = step * one_step + 1
        j = pageNum + 1 
    for page in range(int(i), int(j)):
        try:
            url = 'http://weibo.cn/u/%d?filter=1&page=%d'%(user_id, page)
            lxml = requests.get(url, cookies = cookie).content
            
            selector = etree.HTML(lxml)
            content = selector.xpath('//span[@class = "ctt"]')
            for each in content:
                text = each.xpath('string(.)')
                if word_count >= 3:
                    text = "%d: "%(word_count - 2) + text + "\n"
                else:
                    text = text + "\n\n"
                result = result  + text
                word_count += 1
            print(page, 'word ok')
            sys.stdout.flush()
            soup = BeautifulSoup(lxml, "lxml")
            urllist = soup.find_all('a', href=re.compile(r'^http://weibo.cn/mblog/oripic', re.I))
            urllist1 = soup.find_all('a', href=re.compile(r'^http://weibo.cn/mblog/picAll', re.I))
            for imgurl in urllist:
                imgurl['href'] = re.sub(r"amp;", '', imgurl['href'])
                urllist_set.add(requests.get(imgurl['href'], cookies = cookie).url)
                image_count += 1
            for imgurl_all in urllist1:
                html_content = requests.get(imgurl_all['href'], cookies = cookie).content
                soup = BeautifulSoup(html_content, "lxml")
                urllist2 = soup.find_all('a', href = re.compile(r'^/mblog/oripic', re.I))
                for imgurl in urllist2:
                    imgurl['href'] = 'http://weibo.cn' + re.sub(r"amp;", '', imgurl['href'])
                    urllist_set.add(requests.get(imgurl['href'], cookies = cookie).url)
                    image_count += 1
                image_count -= 1
            print(page, 'picurl ok')
        except:
            print(page, 'error')
        print(page, 'sleep')
        sys.stdout.flush()
        time.sleep(60)
    print(u'正在进行第', step + 1, u'次停顿，防止访问次数过多')
    time.sleep(300)

1 work ok
1 picurl ok
1 sleep
2 work ok
2 picurl ok
2 sleep
3 work ok
3 picurl ok
3 sleep
4 work ok
4 picurl ok
4 sleep
5 work ok
5 picurl ok
5 sleep
6 work ok
6 picurl ok
6 sleep
7 work ok
7 picurl ok
7 sleep
8 work ok
8 picurl ok
8 sleep
9 work ok
9 picurl ok
9 sleep
10 work ok
10 picurl ok
10 sleep
11 work ok
11 picurl ok
11 sleep
12 work ok
12 picurl ok
12 sleep
13 work ok
13 picurl ok
13 sleep
14 work ok
14 picurl ok
14 sleep
15 work ok
15 picurl ok
15 sleep
16 work ok
16 picurl ok
16 sleep
17 work ok
17 picurl ok
17 sleep
18 work ok
18 picurl ok
18 sleep
19 work ok
19 picurl ok
19 sleep
20 work ok
20 picurl ok
20 sleep
21 work ok
21 picurl ok
21 sleep
22 work ok
22 picurl ok
22 sleep
23 work ok
23 picurl ok
23 sleep
24 work ok
24 picurl ok
24 sleep
25 work ok
25 picurl ok
25 sleep
26 work ok
26 picurl ok
26 sleep
27 work ok
27 picurl ok
27 sleep
28 work ok
28 picurl ok
28 sleep
29 work ok
29 picurl ok
29 sleep
30 work ok
30 picurl ok
30 sleep
31 work ok
31 picurl ok
31 sleep
32 w

In [77]:
try: 
    fo = open(os.getcwd() + "/%d"%user_id, "wb")
    fo.write(result.encode())
    word_path = os.getcwd() + '/%d'%user_id 
    print(u'文字微博爬取完畢')
    link = ""
    fo2 = open(os.getcwd() + "/%s_image"%user_id, "wb")
    for eachlink in urllist_set:
        link = link + eachlink + "\n"
        fo2.write(link.encode())
    print(u'圖片鏈接爬取完畢')
except:
    print(u'存放數據地址有誤')
sys.stdout.flush()

文字微博爬取完畢
圖片鏈接爬取完畢


In [87]:
for h in urllist_set:
    print(h)

http://wx4.sinaimg.cn/large/73c7ca2fly1ff2fdd0b31j217e17e10i.jpg
http://wx2.sinaimg.cn/large/73c7ca2fly1fibg6vsuo7j20ku0ku0vo.jpg
http://wx2.sinaimg.cn/large/73c7ca2fly1ff0hol97slj217e17etig.jpg
http://wx2.sinaimg.cn/large/73c7ca2fly1fmqjlt7xwjj20ku0kutmf.jpg
http://wx3.sinaimg.cn/large/73c7ca2fly1ff15glzooxj20ku0kun45.jpg
http://wx1.sinaimg.cn/large/73c7ca2fly1fkdiymuzcej21ej2h7b2b.jpg
http://wx2.sinaimg.cn/large/73c7ca2fly1fn3w9mijikj20ku1127tv.jpg
http://wx4.sinaimg.cn/large/73c7ca2fly1ffwiao8stfj20qo0qo483.jpg
http://wx1.sinaimg.cn/large/73c7ca2fly1fjbaz7mruej20p1142gq3.jpg
http://wx1.sinaimg.cn/large/73c7ca2fgy1fhrqxllemmj20ku0kuwjg.jpg
http://wx4.sinaimg.cn/large/73c7ca2fly1fhmymirikdj20ku0h70t1.jpg
http://wx1.sinaimg.cn/large/73c7ca2fly1fn3w9ddi83j20ku112tnd.jpg
http://wx4.sinaimg.cn/large/73c7ca2fly1ffv310xrgbj20ku0kugo7.jpg
http://wx4.sinaimg.cn/large/73c7ca2fly1fiw8d5omvmj20ku0kun08.jpg
http://wx3.sinaimg.cn/large/73c7ca2fly1fiut8cw3e0j21kw2rcnpe.jpg
http://wx2.sinaimg.cn/lar

In [79]:
len(link)

35815

In [76]:
if not urllist_set:
    print('该用户原创微博中不存在图片')
else:
    image_path = os.getcwd() + '/weibo_image'
    if os.path.exists(image_path) is False:
        os.mkdir(image_path)
    x = 1
    for imgurl in urllist_set:
        temp = image_path + '/%s.jpg' % x
        print('正在下载第%s张图片'%x)
        try:
            r = requests.get(imgurl, stream=True)
            if r.status_code == 200:
                with open(temp, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
        except:
            print('该图片下载失败:%s'%imgurl)
        x += 1
print('原创微博爬取完毕，共%d条，保存路径%s'%(word_count - 3, word_path))
print('微博图片爬取完毕，共%d张，保存路径%s'%(image_count - 1, image_path))

正在下载第1张图片
正在下载第2张图片
正在下载第3张图片
正在下载第4张图片
正在下载第5张图片
正在下载第6张图片
正在下载第7张图片
正在下载第8张图片
正在下载第9张图片
正在下载第10张图片
正在下载第11张图片
正在下载第12张图片
正在下载第13张图片
正在下载第14张图片
正在下载第15张图片
正在下载第16张图片
正在下载第17张图片
正在下载第18张图片
正在下载第19张图片
正在下载第20张图片
正在下载第21张图片
正在下载第22张图片
正在下载第23张图片
正在下载第24张图片
正在下载第25张图片
正在下载第26张图片
正在下载第27张图片
正在下载第28张图片
正在下载第29张图片
正在下载第30张图片
正在下载第31张图片
正在下载第32张图片
正在下载第33张图片
正在下载第34张图片
正在下载第35张图片
正在下载第36张图片
正在下载第37张图片
正在下载第38张图片
正在下载第39张图片
正在下载第40张图片
正在下载第41张图片
正在下载第42张图片
正在下载第43张图片
正在下载第44张图片
正在下载第45张图片
正在下载第46张图片
正在下载第47张图片
正在下载第48张图片
正在下载第49张图片
正在下载第50张图片
正在下载第51张图片
正在下载第52张图片
正在下载第53张图片
正在下载第54张图片
正在下载第55张图片
正在下载第56张图片
正在下载第57张图片
正在下载第58张图片
正在下载第59张图片
正在下载第60张图片
正在下载第61张图片
正在下载第62张图片
正在下载第63张图片
正在下载第64张图片
正在下载第65张图片
正在下载第66张图片
正在下载第67张图片
正在下载第68张图片
正在下载第69张图片
正在下载第70张图片
正在下载第71张图片
正在下载第72张图片
正在下载第73张图片
正在下载第74张图片
正在下载第75张图片
正在下载第76张图片
正在下载第77张图片
正在下载第78张图片
正在下载第79张图片
正在下载第80张图片
正在下载第81张图片
正在下载第82张图片
正在下载第83张图片
正在下载第84张图片
正在下载第85张图片
正在下载第86张图片
正在下载第87张图片
正在下载第88张图片
正在下载第89张图片
正在下载第90张图片
正在下载第91张图片
正在下载第92张

In [82]:
len(urllist_set)

551